## Подготовка

In [1]:
import warnings

warnings.filterwarnings('ignore')

import lightgbm as lgb
import numpy as np
import optuna
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv('APLE_PERIOD_D1.csv', index_col=[0], parse_dates=[0])

In [3]:
data = data.iloc[::-1]
data

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A30,A31,A32,A33,A34,A35,A36,A37,A38,Close
data,,,,,,,,,,,,,,,,,,,,,
2017-11-30,0.035,-0.015,0.0875,0.0075,-0.0175,-0.0375,0.0875,0.0075,1.000000e-02,7.105427e-15,...,-0.06,0.000050,-0.029931,0.000000,0.000000,0.03,0.03,3.552714e-15,-0.037431,1
2017-12-01,-0.015,-0.045,0.0075,0.0225,-0.0375,-0.1125,0.0075,0.0225,7.105427e-15,-6.000000e-02,...,-0.15,-0.029931,0.000069,0.000000,0.000000,0.00,0.00,1.500000e-01,-0.022431,0
2017-12-04,-0.045,0.060,0.0225,0.1500,-0.1125,-0.0300,0.0225,0.1500,-6.000000e-02,7.500000e-02,...,-0.03,0.000069,-0.029912,0.000000,0.000000,0.03,0.03,3.750000e-02,-0.179912,1
2017-12-05,0.060,-0.060,0.1500,0.0300,-0.0300,-0.1500,0.1500,0.0300,7.500000e-02,-7.500000e-02,...,-0.09,-0.029912,0.000088,0.000000,0.000000,0.00,0.00,1.875000e-01,-0.029912,0
2017-12-06,-0.060,0.045,0.0300,0.1125,-0.1500,-0.0225,0.0300,0.1125,-7.500000e-02,4.500000e-02,...,0.00,0.000088,0.000088,0.000000,0.000000,0.00,0.00,2.250000e-02,-0.112412,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-15,0.075,0.165,0.1875,0.4125,-0.0375,-0.0825,0.3250,0.2375,1.350000e-01,1.100000e-01,...,-0.24,0.026316,0.056391,0.094411,0.014286,-0.09,0.32,6.928571e-02,-0.356109,0
2024-03-18,0.165,0.070,0.4125,0.1750,-0.0825,-0.0350,0.2375,0.1500,1.100000e-01,6.500000e-02,...,-0.12,0.056391,-0.033477,0.014286,-0.085777,0.02,0.31,-5.327694e-02,-0.208477,1
2024-03-19,0.070,-0.095,0.1750,0.0475,-0.0350,-0.2375,0.1500,0.0725,6.500000e-02,-5.000000e-02,...,-0.31,-0.033477,-0.183252,-0.085777,-0.145802,0.27,0.40,-2.080201e-02,-0.230752,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1586 entries, 2017-11-30 to 2024-03-21
Data columns (total 39 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A1      1586 non-null   float64
 1   A2      1586 non-null   float64
 2   A3      1586 non-null   float64
 3   A4      1586 non-null   float64
 4   A5      1586 non-null   float64
 5   A6      1586 non-null   float64
 6   A7      1586 non-null   float64
 7   A8      1586 non-null   float64
 8   A9      1586 non-null   float64
 9   A10     1586 non-null   float64
 10  A11     1586 non-null   float64
 11  A12     1586 non-null   float64
 12  A13     1586 non-null   float64
 13  A14     1586 non-null   float64
 14  A15     1586 non-null   float64
 15  A16     1586 non-null   float64
 16  A17     1586 non-null   float64
 17  A18     1586 non-null   float64
 18  A19     1586 non-null   float64
 19  A20     1586 non-null   float64
 20  A21     1586 non-null   float64
 21  A22     1586 non-nu

## Обучение

In [5]:
X = data.drop('Close', axis=1)
y = data[['Close']]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, random_state=42)

In [7]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
pca = PCA(n_components=7)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [9]:
pca.explained_variance_ratio_

array([0.57315351, 0.09923983, 0.08368575, 0.07269912, 0.03499105,
       0.02666934, 0.02303646])

In [10]:
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
print("Суммарная доля объяснённой дисперсии:", cumulative_variance[-1])

Суммарная доля объяснённой дисперсии: 0.9134750655407532


#### Модель LightGBM

In [11]:
train_set = lgb.Dataset(X_train, label=y_train)
test_set = lgb.Dataset(X_test, label=y_test, reference=train_set)
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'verbose': -1
}
early_stopping_cb = lgb.early_stopping(stopping_rounds=50)
model = lgb.train(
    params=params,
    train_set=train_set,
    num_boost_round=500,
    valid_sets=[train_set, test_set],
    valid_names=['train', 'valid'],
    callbacks=[early_stopping_cb, lgb.log_evaluation(10)]
)
y_pred = model.predict(X_test)
y_pred_binary = [1 if prob > 0.5 else 0 for prob in y_pred]

Training until validation scores don't improve for 50 rounds
[10]	train's binary_logloss: 0.562629	valid's binary_logloss: 0.632678
[20]	train's binary_logloss: 0.483412	valid's binary_logloss: 0.617783
[30]	train's binary_logloss: 0.425084	valid's binary_logloss: 0.618664
[40]	train's binary_logloss: 0.373981	valid's binary_logloss: 0.621302
[50]	train's binary_logloss: 0.329234	valid's binary_logloss: 0.627306
[60]	train's binary_logloss: 0.291211	valid's binary_logloss: 0.638099
[70]	train's binary_logloss: 0.258406	valid's binary_logloss: 0.647431
Early stopping, best iteration is:
[27]	train's binary_logloss: 0.441735	valid's binary_logloss: 0.617009


In [12]:
print(classification_report(y_test, y_pred_binary))

              precision    recall  f1-score   support

           0       0.71      0.70      0.71       200
           1       0.70      0.71      0.70       197

    accuracy                           0.71       397
   macro avg       0.71      0.71      0.71       397
weighted avg       0.71      0.71      0.71       397



#### Подбор параметров

In [13]:
def objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'precision',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'random_state': 42
    }
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return precision_score(y_test, y_pred, average='weighted')

In [14]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)

In [15]:
best_params = study.best_params
print(best_params)

{'num_leaves': 148, 'learning_rate': 0.010207755877428068, 'max_depth': 3, 'n_estimators': 186}


In [16]:
best_params.update({
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
})

#### Обучение с лучшими параметрами

In [17]:
model_best = lgb.train(
    params=best_params,
    train_set=train_set,
    num_boost_round=500,
    valid_sets=[train_set, test_set],
    valid_names=['train', 'valid'],
    callbacks=[early_stopping_cb, lgb.log_evaluation(10)]
)
y_pred_best = model_best.predict(X_test)
y_pred_binary_best = [1 if prob > 0.5 else 0 for prob in y_pred_best]

Training until validation scores don't improve for 50 rounds
[10]	train's binary_logloss: 0.674266	valid's binary_logloss: 0.678678
[20]	train's binary_logloss: 0.657982	valid's binary_logloss: 0.66623
[30]	train's binary_logloss: 0.644061	valid's binary_logloss: 0.65573
[40]	train's binary_logloss: 0.632032	valid's binary_logloss: 0.646854
[50]	train's binary_logloss: 0.621436	valid's binary_logloss: 0.639611
[60]	train's binary_logloss: 0.612153	valid's binary_logloss: 0.633503
[70]	train's binary_logloss: 0.603926	valid's binary_logloss: 0.627939
[80]	train's binary_logloss: 0.596468	valid's binary_logloss: 0.623316
[90]	train's binary_logloss: 0.589607	valid's binary_logloss: 0.620328
[100]	train's binary_logloss: 0.583217	valid's binary_logloss: 0.617306
[110]	train's binary_logloss: 0.577404	valid's binary_logloss: 0.614691
[120]	train's binary_logloss: 0.572131	valid's binary_logloss: 0.612431
[130]	train's binary_logloss: 0.567175	valid's binary_logloss: 0.610491
[140]	train's 

In [18]:
print(classification_report(y_test, y_pred_binary_best))

              precision    recall  f1-score   support

           0       0.74      0.71      0.73       200
           1       0.72      0.75      0.73       197

    accuracy                           0.73       397
   macro avg       0.73      0.73      0.73       397
weighted avg       0.73      0.73      0.73       397

